In [1]:
import torch
import pickle
import numpy as np
from argparse import Namespace
from torch.utils.data import DataLoader
import torch.nn.functional as F
from heterogt.utils.tokenizer import EHRTokenizer
from heterogt.utils.dataset import FineTuneEHRDataset, batcher, expand_level3
from heterogt.utils.train import train_with_early_stopping
from heterogt.utils.seed import set_random_seed
from heterogt.model.model import HeteroGTFineTune

Disabling PyTorch because PyTorch >= 2.1 is required but found 1.13.1
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
set_random_seed(123)

[INFO] Random seed set to 123


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
config = Namespace(
    dataset = "MIMIC-III",
    tasks = ["death", "readmission", "stay", "next_diag_6m", "next_diag_12m"], 
    task_index = 2,  # index of the task to train
    token_type = ["diag", "med", "lab", "pro"],
    special_tokens = ["[PAD]", "[CLS]"],
    attn_mask_dicts = [{1:[6,7], 2:[6,7], 3:[6,7], 4:[6,7], 6:[2,3,4,5,6,7], 7:[2,3,4,5,6,7]}, 
                       {1:[6,7], 2:[6,7], 3:[6,7], 4:[6,7], 6:[2,3,4,5,6,7], 7:[2,3,4,5,6,7]}],
    d_model = 64,
    num_heads = 4,
    batch_size = 32,
    lr = 1e-3,
    epochs = 500,
    early_stop_patience = 5,
    group_code_thre = 5,  # if there are group_code_thre diag codes belongs to the same group ICD code, then the group code is generated
    use_pretrained_model = True,
    pretrain_mask_rate = 0.7,
    pretrain_cls_ontology_weight = 5e-2,
    pretrain_visit_ontology_weight = 5e-2,
    pretrain_adm_type_weight = 5e-2,
)

In [5]:
full_data_path = f"./data_process/{config.dataset}-processed/mimic.pkl"  # for tokenizer
curr_task = config.tasks[config.task_index]
print("Current task:", curr_task)
if curr_task == "next_diag_6m":
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_nextdiag_6m.pkl"
elif curr_task == "next_diag_12m":
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_nextdiag_12m.pkl"
else:
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_downstream.pkl"

Current task: stay


In [6]:
ehr_full_data = pickle.load(open(full_data_path, 'rb'))
group_code_sentences = [expand_level3()[1]]
diag_sentences = ehr_full_data["ICD9_CODE"].values.tolist()
med_sentences = ehr_full_data["NDC"].values.tolist()
lab_sentences = ehr_full_data["LAB_TEST"].values.tolist()
pro_sentences = ehr_full_data["PRO_CODE"].values.tolist()
age_sentences = [[str(c)] for c in set(ehr_full_data["AGE"].values.tolist())] # important of [[]]
adm_type_sentences = ehr_full_data["ADMISSION_TYPE"].values.tolist()
max_admissions = ehr_full_data.groupby("SUBJECT_ID")["HADM_ID"].nunique().max()
config.max_num_adms = max_admissions
print(f"Max admissions per patient: {config.max_num_adms}")

Max admissions per patient: 8


In [7]:
tokenizer = EHRTokenizer(age_sentences, group_code_sentences, diag_sentences, med_sentences, lab_sentences, 
                         pro_sentences, special_tokens=config.special_tokens, adm_types_sentences=adm_type_sentences)
config.label_vocab_size = len(tokenizer.diag_voc.id2word)  # only for diagnosis
config.global_vocab_size = len(tokenizer.vocab.id2word)
config.age_vocab_size = tokenizer.token_number("age")
config.group_code_vocab_size = tokenizer.token_number("group")
print(f"Age vocabulary size: {config.age_vocab_size}")
print(f"Group code vocabulary size: {config.group_code_vocab_size}")

Age vocabulary size: 18
Group code vocabulary size: 19


In [8]:
train_data, val_data, test_data = pickle.load(open(finetune_data_path, 'rb'))
# example label percentage
print("Percentage of DEATH in test dataset:",
      (test_data["DEATH"] == True).mean() * 100, "%")

print("Percentage of READMISSION in test dataset:",
      (test_data["READMISSION"] == 1).mean() * 100, "%")

print("Percentage of STAY>7 days in test dataset:",
      (test_data["STAY_DAYS"] > 7).mean() * 100, "%")

Percentage of DEATH in test dataset: 28.648477157360407 %
Percentage of READMISSION in test dataset: 40.1491116751269 %
Percentage of STAY>7 days in test dataset: 50.58692893401015 %


In [9]:
train_dataset = FineTuneEHRDataset(train_data, tokenizer, token_type=config.token_type, task=curr_task, 
                                   max_num_adms=config.max_num_adms, group_code_thre=config.group_code_thre)
val_dataset = FineTuneEHRDataset(val_data, tokenizer, token_type=config.token_type, task=curr_task, 
                                 max_num_adms=config.max_num_adms, group_code_thre=config.group_code_thre)
test_dataset = FineTuneEHRDataset(test_data, tokenizer, token_type=config.token_type, task=curr_task, 
                                   max_num_adms=config.max_num_adms, group_code_thre=config.group_code_thre)

In [10]:
num_group_code = []
for i in range(len(train_dataset)):
    input_ids, token_types, adm_index, age_ids, diag_group_codes, labels = train_dataset[i]
    count = (token_types[0] == 6).sum().item()
    num_group_code.append(count)
print("Mean group token numer per patient", np.mean(num_group_code))

Mean group token numer per patient 0.7971893963589908


In [11]:
train_dataloader = DataLoader(
    train_dataset,
    collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=True,
    batch_size=config.batch_size,
)

val_dataloader = DataLoader(
    val_dataset,
    collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=False,
    batch_size=config.batch_size,
)

test_dataloader = DataLoader(
    test_dataset,
    collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=False,
    batch_size=config.batch_size,
)

In [12]:
for batch in train_dataloader:
    pass  # just to check if the dataloader works
for batch in val_dataloader:
    pass  # just to check if the dataloader works
for batch in test_dataloader:
    pass  # just to check if the dataloader works
print("All pass!")

All pass!


In [13]:
if curr_task in ["death", "stay", "readmission"]:
    eval_metric = "f1"
    task_type = "binary"
    loss_fn = F.binary_cross_entropy_with_logits
else:
    eval_metric = "f1"
    task_type = "l2r"
    loss_fn = lambda x, y: F.binary_cross_entropy_with_logits(x, y)

In [14]:
input_ids, token_types, adm_index, age_ids, diag_code_group_dicts, labels = next(iter(train_dataloader))
print("Input IDs shape:", input_ids.shape)
print("Token Types shape:", token_types.shape)
print("Admission Index shape:", adm_index.shape)
print("Age IDs shape:", age_ids.shape)
print("Diag Code Group Dict number:", len(diag_code_group_dicts))
print("Labels shape:", labels.shape)

Input IDs shape: torch.Size([32, 293])
Token Types shape: torch.Size([32, 293])
Admission Index shape: torch.Size([32, 293])
Age IDs shape: torch.Size([32, 8])
Diag Code Group Dict number: 32
Labels shape: torch.Size([32, 1])


# Model Walkthrough

In [15]:
# load pretrained model
if config.use_pretrained_model:
    pretrain_exp_name = (
    f"{config.dataset}-{config.pretrain_mask_rate}-{config.d_model}-{config.pretrain_cls_ontology_weight}-{config.pretrain_visit_ontology_weight}-{config.pretrain_adm_type_weight}"
)
    print(pretrain_exp_name)
    save_path = "./pretrained_models/" + pretrain_exp_name
    state_dict = torch.load(f"{save_path}/pretrained_model.pt", map_location="cpu")

MIMIC-III-0.7-64-0.05-0.05-0.05


In [16]:
final_metrics = []
for i in range(15):
    model = HeteroGTFineTune(tokenizer=tokenizer, token_types=config.token_type, d_model=config.d_model, num_heads=config.num_heads, layer_types=['gnn', 'tf', 'gnn', 'tf'], max_num_adms=config.max_num_adms, 
                     device=device, task=curr_task, label_vocab_size=config.label_vocab_size, attn_mask_dicts=config.attn_mask_dicts,
                     use_cls_cat=True).to(device)
    if config.use_pretrained_model:
        model.load_weight(state_dict)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr)
    best_test_metric = train_with_early_stopping(model, train_dataloader, val_dataloader, test_dataloader,
                                             optimizer, loss_fn, device, config.early_stop_patience, task_type, config.epochs, 
                                             val_long_seq_idx=None, test_long_seq_idx=None, eval_metric=eval_metric, return_model=False)
    final_metrics.append(best_test_metric)

[Warning] Missing keys: ['cls_head.cls.0.weight', 'cls_head.cls.0.bias', 'cls_head.cls.2.weight', 'cls_head.cls.2.bias']
[Warning] Unexpected keys: ['diag_cls.cls.0.weight', 'diag_cls.cls.0.bias', 'diag_cls.cls.2.weight', 'diag_cls.cls.2.bias', 'med_cls.cls.0.weight', 'med_cls.cls.0.bias', 'med_cls.cls.2.weight', 'med_cls.cls.2.bias', 'lab_cls.cls.0.weight', 'lab_cls.cls.0.bias', 'lab_cls.cls.2.weight', 'lab_cls.cls.2.bias', 'pro_cls.cls.0.weight', 'pro_cls.cls.0.bias', 'pro_cls.cls.2.weight', 'pro_cls.cls.2.bias', 'cls_ontology.cls.0.weight', 'cls_ontology.cls.0.bias', 'cls_ontology.cls.2.weight', 'cls_ontology.cls.2.bias', 'visit_ontology.cls.0.weight', 'visit_ontology.cls.0.bias', 'visit_ontology.cls.2.weight', 'visit_ontology.cls.2.bias', 'adm_type.cls.0.weight', 'adm_type.cls.0.bias', 'adm_type.cls.2.weight', 'adm_type.cls.2.bias']


Running inference: 100%|██████████| 197/197 [00:12<00:00, 16.25it/s]


Validation: {'precision': 0.6792074241267139, 'recall': 0.8491690184984347, 'f1': 0.7547380106673091, 'auc': 0.822666007158338, 'prauc': 0.830854807705752}
Test:      {'precision': 0.6826633165811994, 'recall': 0.8519912198154532, 'f1': 0.7579857671329643, 'auc': 0.818919419533539, 'prauc': 0.8261645372203121}


Running inference: 100%|██████████| 197/197 [00:09<00:00, 21.69it/s]


Validation: {'precision': 0.7083333333314169, 'recall': 0.8209470053282504, 'f1': 0.7604938221853977, 'auc': 0.8320041688481985, 'prauc': 0.8438063316112501}
Test:      {'precision': 0.7047593439077581, 'recall': 0.8218877391005899, 'f1': 0.7588303366601243, 'auc': 0.8313457123629732, 'prauc': 0.8430718156633308}


...

Validation: {'precision': 0.7797494780766189, 'recall': 0.7027281279375894, 'f1': 0.739237996000667, 'auc': 0.8344865184099177, 'prauc': 0.8421466027332976}
Test:      {'precision': 0.7737931034456076, 'recall': 0.7036688617099289, 'f1': 0.7370668368547943, 'auc': 0.8351498706176479, 'prauc': 0.8417431610346554}


Running inference: 100%|██████████| 197/197 [00:09<00:00, 21.23it/s]


Validation: {'precision': 0.7466747279300282, 'recall': 0.7745374725595029, 'f1': 0.7603509261983518, 'auc': 0.8349531677750405, 'prauc': 0.839678146743156}
Test:      {'precision': 0.7315416420533623, 'recall': 0.776732518028295, 'f1': 0.7534600710478241, 'auc': 0.8313837645155624, 'prauc': 0.835487608027023}


Running inference: 100%|██████████| 197/197 [00:09<00:00, 21.16it/s]

Validation: {'precision': 0.7089947089927346, 'recall': 0.798369394792103, 'f1': 0.7510324433929436, 'auc': 0.8238656612480281, 'prauc': 0.826945654394171}
Test:      {'precision': 0.7093690248546589, 'recall': 0.8143618689218741, 'f1': 0.7582481702040076, 'auc': 0.8257189264662286, 'prauc': 0.8303550346320139}

Early stopping triggered (no improvement for 5 epochs).

Best validation performance:
{'precision': 0.7388792605409045, 'recall': 0.8021323298814609, 'f1': 0.7692076329552916, 'auc': 0.8440810885785797, 'prauc': 0.8542849986126606}
Corresponding test performance:
{'precision': 0.7358708189136798, 'recall': 0.8002508623367819, 'f1': 0.7667117270178957, 'auc': 0.8382803648375964, 'prauc': 0.8488566327870246}


In [17]:
def topk_avg_performance_formatted(performances, k=5):
    metrics = ["f1", "auc", "prauc"]
    scores = {m: np.array([p[m] for p in performances]) for m in metrics}

    # 计算排名（值越大排名越靠前）
    ranks = {m: (-scores[m]).argsort().argsort() + 1 for m in metrics}
    avg_ranks = np.mean(np.stack([ranks[m] for m in metrics], axis=1), axis=1)

    # 选 top-k
    topk_idx = np.argsort(avg_ranks)[:k]
    final_avg = {m: np.mean([performances[i][m] for i in topk_idx]) for m in performances[0].keys()}
    final_std = {m: np.std([performances[i][m] for i in topk_idx], ddof=0) for m in performances[0].keys()}

    # 打印结果
    print("Final Metrics:")
    for m in performances[0].keys():
        print(f"{m}: {final_avg[m]:.4f}±{final_std[m]:.4f}")

In [18]:
topk_avg_performance_formatted(final_metrics, 5)

Final Metrics:
precision: 0.7164±0.0171
recall: 0.8275±0.0226
f1: 0.7674±0.0029
auc: 0.8394±0.0016
prauc: 0.8480±0.0023
